# 🩺 Clasificación de Melanomas a partir de Imágenes

## 🌑 Introducción
El **melanoma** es un tipo de cáncer de piel que se origina en los **melanocitos**, las células encargadas de producir la melanina, el pigmento que da color a la piel, ojos y cabello.  
Aunque representa un porcentaje pequeño de los cánceres de piel, es el **más agresivo** debido a su gran capacidad para invadir otros tejidos y producir metástasis.  
La detección temprana es fundamental, ya que aumenta considerablemente las posibilidades de un tratamiento exitoso y de supervivencia del paciente.  

---

## 🔎 Metodología clínica: Regla ABCD
En dermatología se utiliza la **regla ABCD** como guía visual para identificar posibles melanomas:  

- **A – Asimetría**: los melanomas suelen tener formas irregulares; un lunar benigno suele ser simétrico.  
- **B – Bordes**: los melanomas presentan bordes difusos, dentados o irregulares, frente a los bordes definidos de los lunares normales.  
- **C – Color**: los melanomas pueden mostrar múltiples tonalidades (negro, marrón, rojo, azul, blanco), mientras que los lunares benignos suelen ser uniformes.  
- **D – Diámetro**: lesiones con un diámetro superior a 6 mm se consideran sospechosas.  

A esta regla a veces se añade la **E – Evolución**, que hace referencia a cambios en el tamaño, forma, color o la aparición de síntomas (picor, sangrado).  

---

## 🎯 Objetivo del trabajo
El objetivo de este proyecto es **desarrollar un modelo de aprendizaje automático capaz de clasificar imágenes de lesiones cutáneas** para distinguir entre melanomas y lesiones benignas.  
Para ello:  
- Se trabajará con un conjunto de imágenes médicas.  
- Se aplicará un flujo de trabajo basado en la preparación y preprocesamiento de datos, entrenamiento de modelos de deep learning y análisis de resultados.  
- Se evaluará el desempeño de los modelos con métricas adecuadas, y se comparará con la metodología clínica tradicional basada en la regla ABCD.  

Este proyecto busca **apoyar el diagnóstico médico mediante técnicas computacionales**, aportando una herramienta complementaria a la práctica clínica.


# IMPORTACIONES Y RUTAS NECESARIAS

In [1]:
# IMPORTACIONES
# Standard library
from pathlib import Path
import shutil
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np
from PIL import Image, ImageOps
import cv2
from skimage import color, filters, morphology, util

# Third-party
import torch
from PIL import Image, ImageOps
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import functional as TF


# RUTAS

BASE_DIR = Path(".").resolve()
DATA_DIR = Path("D:/proyectos/Caso_aprendizaje-Melanomas/data/raw")
SPLITS = ["train", "val", "test"]
ALLOWED_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
TRAIN_DIR = DATA_DIR / "train"
VAL_DIR   = DATA_DIR / "val"
TEST_DIR  = DATA_DIR / "test"
QUAR_DIR = DATA_DIR / "_quarantine"
QUAR_DIR.mkdir(exist_ok=True)
for p in (TRAIN_DIR, VAL_DIR, TEST_DIR): print(p, "exists:", p.exists())

# Hiperparámetros
CLASSES = 2
BATCH   = 32
ROWS = COLS = 224
INPUT_CH = 3
EPOCHS  = 20
TEST_MAX_SAMPLES = 3000
SEED    = 42


OSError: [WinError 1455] El archivo de paginación es demasiado pequeño para completar la operación. Error loading "d:\proyectos\Caso_aprendizaje-Melanomas\.venv\lib\site-packages\torch\lib\cublas64_12.dll" or one of its dependencies.

## 📌 ¿Por qué separar en *train*, *validation* y *test*?

En proyectos de *Machine Learning* es fundamental dividir los datos en diferentes conjuntos:

- **Train (entrenamiento)**  
  Se utiliza para que el modelo aprenda los patrones de las imágenes.  

- **Validation (validación)**  
  Sirve para ajustar hiperparámetros (número de capas, tasa de aprendizaje, etc.) y comparar modelos sin mirar el *test*.  
  👉 Es una especie de "examen parcial": nos avisa si el modelo se está sobreajustando (*overfitting*) o si generaliza bien.  

- **Test (prueba final)**  
  Es el conjunto que se mantiene completamente aislado durante todo el desarrollo.  
  👉 Representa el "examen final" y nos da una estimación real del rendimiento del modelo en datos nunca vistos.  

✅ En este caso (clasificación de melanomas), separar en *train* y *val* es crucial porque:  
- Ayuda a evitar *overfitting*, que sería muy peligroso si el modelo solo memoriza imágenes.  
- Permite evaluar de manera más realista la capacidad de generalización antes de aplicarlo en un entorno clínico.  


In [ ]:
# Función para contar imágenes por carpeta
def count_images_in_dir(dir_path: Path):
    if not dir_path.exists():
        return {}
    counts = {}
    for cls_dir in dir_path.iterdir():
        if cls_dir.is_dir():
            n_imgs = sum(1 for f in cls_dir.rglob("*") if f.suffix.lower() in ALLOWED_EXTS)
            counts[cls_dir.name] = n_imgs
    return counts

# Recolectar conteos
all_counts = {}
for split in SPLITS:
    split_dir = DATA_DIR / split
    all_counts[split] = count_images_in_dir(split_dir)

# Mostrar tabla
import pandas as pd

df = pd.DataFrame(all_counts).fillna(0).astype(int).T
df["Total"] = df.sum(axis=1)
display(df)



,Benign,Malignant,Total
train,5346,4752,10098
val,943,838,1781
test,1000,1000,2000


## ⚡ Uso de GPU en PyTorch

Para acelerar el entrenamiento de los modelos, es importante usar la **GPU** en lugar de la **CPU** siempre que esté disponible.  

Con PyTorch podemos:  
- Detectar automáticamente si el sistema tiene una GPU con CUDA.  
- Mover el **modelo** y los **tensores** al dispositivo adecuado (`cuda` o `cpu`).  
- Asegurarnos de que todo el flujo de entrenamiento (inputs, labels, modelo, pérdidas) se ejecute en la GPU.  

El siguiente código hace esta detección y muestra qué dispositivo se está utilizando.


In [ ]:
import torch

print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# prueba rápida en GPU
x = torch.randn(4096, 4096, device=device)
y = x @ x
print("Tensor en:", y.device)


CUDA disponible: True
GPU: NVIDIA GeForce MX250
Tensor en: cuda:0


## 🖼️ Dimensionado consistente de imágenes

Para que el modelo funcione con cualquier imagen, todas deben tener **mismo tamaño** y **misma normalización**.

- Tamaño objetivo: `224x224` (estándar para muchas CNN).
- Estrategia:
  - **Entrenamiento**: redimensionar + (opcional) aumentos de datos ligeros.
  - **Validación/Test**: solo redimensionar (sin aumentos).
  - **Inferencia**: usar **exactamente** la misma trasformación que `val/test`.

> Nota: es preferible **transformar al vuelo** (con `DataLoader`) y **no sobrescribir** las imágenes en disco. Si quieres un *export* a `data/processed/`, al final tienes una celda opcional para guardar las redimensionadas.


## A) Limpieza rápida (corruptas + orientación + RGB)

In [ ]:

def clean_folder(split_dir: Path):
    n_total=n_quar=0
    for p in split_dir.rglob("*"):
        if p.suffix.lower() not in {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}:
            continue
        n_total += 1
        try:
            with Image.open(p) as im:
                im = ImageOps.exif_transpose(im)   # respeta la orientación EXIF
                if im.mode != "RGB":
                    im = im.convert("RGB")         # fuerza 3 canales
                # No se guarda en disco: solo validamos que abre bien
        except Exception as e:
            n_quar += 1
            shutil.move(str(p), QUAR_DIR / p.name)
    print(f"{split_dir.name}: {n_total} imgs, movidas a cuarentena: {n_quar}")

for split in ["train","val","test"]:
    d = DATA_DIR / split
    if d.exists():
        clean_folder(d)
print("✅ Limpieza básica terminada (sin modificar imágenes buenas).")


train: 10098 imgs, movidas a cuarentena: 0
val: 1781 imgs, movidas a cuarentena: 0
test: 2000 imgs, movidas a cuarentena: 0
✅ Limpieza básica terminada (sin modificar imágenes buenas).


## B) Transforms coherentes (letterbox + resize + tensor)

In [ ]:

class PadToSquare:
    def __init__(self, fill=0): self.fill = fill
    def __call__(self, img):
        w, h = img.size
        s = max(w, h)
        pad_left  = (s - w) // 2
        pad_right = s - w - pad_left
        pad_top   = (s - h) // 2
        pad_bottom= s - h - pad_top
        return TF.pad(img, [pad_left, pad_top, pad_right, pad_bottom], fill=self.fill)

# >>> Si vas a usar TRANSFER LEARNING (recomendado):
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

train_tfms = transforms.Compose([
    PadToSquare(fill=0),
    transforms.Resize((ROWS, COLS)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.05,0.05)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

eval_tfms = transforms.Compose([
    PadToSquare(fill=0),
    transforms.Resize((ROWS, COLS)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


train_ds = datasets.ImageFolder(DATA_DIR / "train", transform=train_tfms)
val_ds   = datasets.ImageFolder(DATA_DIR / "val",   transform=eval_tfms) if (DATA_DIR / "val").exists() else None
test_ds  = datasets.ImageFolder(DATA_DIR / "test",  transform=eval_tfms) if (DATA_DIR / "test").exists() else None

train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=0, pin_memory=(device.type=="cuda"))
val_dl   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda")) if val_ds else None
test_dl  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda")) if test_ds else None

xb, yb = next(iter(train_dl))
print("Batch:", xb.shape)  # [B, 3, 224, 224]


Batch: torch.Size([32, 3, 224, 224])


In [ ]:
# ===== 1.1 Semillas fijas =====
import torch, random, numpy as np
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)


In [ ]:
# ===== 1.2 EXIF dentro del transform =====
from PIL import ImageOps
class ExifTranspose:
    def __call__(self, img):
        return ImageOps.exif_transpose(img)


In [ ]:
# ===== 1.3 Conteo por clase =====
from collections import Counter
def count_per_class(ds):
    idxs = [y for _, y in ds.samples] if hasattr(ds, "samples") else [y for _, y in ds.imgs]
    c = Counter(idxs)
    classes = ds.classes
    return {classes[i]: c.get(i,0) for i in range(len(classes))}

print("Train:", count_per_class(train_ds))
if val_ds:  print("Val:",   count_per_class(val_ds))
if test_ds: print("Test:",  count_per_class(test_ds))


## 2) Padding “inteligente” (evitar bordes negros que meten sesgo)

Tu PadToSquare(fill=0) pone bordes negros. En dermatoscopía esos bordes pueden convertirse en señal espuria. Mejor:

- Reflect (espejado) o

- Rellenar con el color mediano del borde.

In [ ]:
import torchvision.transforms.functional as TF
class PadToSquareReflect:
    def __call__(self, img):
        w, h = img.size
        s = max(w, h)
        pad_left  = (s - w) // 2
        pad_right = s - w - pad_left
        pad_top   = (s - h) // 2
        pad_bottom= s - h - pad_top
        return TF.pad(img, [pad_left, pad_top, pad_right, pad_bottom], padding_mode="reflect")


## 4) Augmentaciones acordes a dermatoscopía

- Rotación: en lesiones de piel la orientación no importa → rotaciones moderadas (±15–30º) OK.

- Flip horizontal: suele estar bien. El flip vertical también es razonable (criterio clínico: no hay “arriba/abajo”).

- Traslación leve: ✔

Evita cambios de color agresivos (jitter fuerte) si harás análisis de color; si usas un modelo de clasificación general, un ColorJitter suave puede ayudar, pero con cuidado.

In [ ]:
train_tfms = transforms.Compose([
    ExifTranspose(),
    PadToSquareReflect(),
    transforms.Resize((ROWS, COLS), antialias=True),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.05,0.05)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),  # o DATA_MEAN/STD
])

eval_tfms = transforms.Compose([
    ExifTranspose(),
    PadToSquareReflect(),
    transforms.Resize((ROWS, COLS), antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])


## 5) DataLoader estable y eficiente

- En Windows, si num_workers>0, añade persistent_workers=True para evitar re-forks.

- Usa pin_memory=(device.type=="cuda").

- Si ves cuellos de botella, sube a num_workers=2–4.

In [ ]:
workers = 2
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True,
                      num_workers=workers, pin_memory=(device.type=="cuda"),
                      persistent_workers=(workers>0))
val_dl   = DataLoader(val_ds, batch_size=BATCH, shuffle=False,
                      num_workers=workers, pin_memory=(device.type=="cuda"),
                      persistent_workers=(workers>0)) if val_ds else None
test_dl  = DataLoader(test_ds, batch_size=BATCH, shuffle=False,
                      num_workers=workers, pin_memory=(device.type=="cuda"),
                      persistent_workers=(workers>0)) if test_ds else None


## 6) “Sanity checks” visuales

Antes de entrenar, mira un batch para comprobar que el padding/rotación/color se ve razonable.

In [ ]:

xb, yb = next(iter(train_dl))
grid = make_grid(xb[:16], nrow=8, normalize=True)  # normalize=True solo para ver
plt.figure(figsize=(12,4))
plt.imshow(grid.permute(1,2,0).cpu().numpy())
plt.axis("off"); plt.title("Muestra de augmentaciones (train)")
plt.show()


## 8) Cosas específicas de piel

- Hair removal (eliminación de pelos) o máscara rápida de lesión antes de redimensionar (evita que el fondo/piel dominen).

- Color constancy (p. ej., Shades-of-Gray) si el color es determinante.

- Recorte por máscara (crop alrededor de la lesión + padding reflect) para centrar la ROI.

In [ ]:
# === Celda única: Preprocesado dermatoscópico (hair removal + color constancy + máscara + crop reflect) ===
# Requisitos: pillow, numpy, opencv-python, scikit-image
# pip install pillow numpy opencv-python scikit-image



# ---------- 1) Hair removal (inpainting sobre trazos oscuros finos) ----------
def hair_removal_bh_inpaint(rgb, se_size=9, thresh_percentile=90, dilate_iter=1, method=cv2.INPAINT_TELEA):
    """
    Elimina pelos oscuros:
    - Black-hat morfológico para detectar filamentos oscuros
    - Umbral por percentil (robusto)
    - Dilatación leve y 'inpainting' para rellenar
    """
    bgr = rgb[:, :, ::-1].copy()  # PIL RGB -> OpenCV BGR
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    # black-hat para resaltar trazos oscuros finos
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (se_size, se_size))
    bh = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # umbral robusto por percentil para seleccionar pelos
    thr = np.percentile(bh, thresh_percentile)
    mask = (bh > thr).astype(np.uint8) * 255
    if dilate_iter > 0:
        mask = cv2.dilate(mask, np.ones((3,3), np.uint8), iterations=dilate_iter)

    # inpainting
    inpainted = cv2.inpaint(bgr, mask, 3, method)
    out = inpainted[:, :, ::-1]  # back to RGB
    return out

# ---------- 2) Color constancy (Shades-of-Gray) ----------
def shades_of_gray_cc(rgb, p=6, eps=1e-6):
    """
    Shades-of-Gray (p-norm). Normaliza ganancias de canal para reducir dominantes de iluminación.
    No cambia el contraste local, solo reescala canales.
    """
    img = rgb.astype(np.float32) / 255.0
    # evitar saturaciones por valores muy cercanos a 0
    img = np.clip(img, 0.0, 1.0) + eps

    # medias p-norm por canal
    mean_p = (img ** p).mean(axis=(0, 1)) ** (1.0 / p)
    # ganancias inversas normalizadas
    gain = (1.0 / mean_p)
    gain = gain / (np.linalg.norm(gain) + eps) * np.sqrt(3)

    out = img * gain[None, None, :]
    out = np.clip(out, 0.0, 1.0)
    return (out * 255.0).astype(np.uint8)

# ---------- 3) Máscara rápida de la lesión (umbral en canal 'a' de CIELAB) ----------
def quick_lesion_mask(rgb, min_obj=300, hole_area=800):
    """
    Segmentación rápida: umbral de Otsu sobre canal 'a' en CIELAB (con blur).
    Limpieza morfológica ligera.
    """
    lab = color.rgb2lab(rgb).astype(np.float32)
    a = lab[:, :, 1]
    a_blur = cv2.GaussianBlur(a, (5, 5), 0)
    th = filters.threshold_otsu(a_blur)
    mask = (a_blur > th)

    # morfología
    mask = morphology.remove_small_holes(mask, area_threshold=hole_area)
    mask = morphology.remove_small_objects(mask, min_size=min_obj)
    mask = morphology.binary_opening(mask, morphology.disk(3))
    mask = morphology.binary_closing(mask, morphology.disk(5))
    return mask.astype(np.uint8)

# ---------- 4) Crop por máscara + reflect pad a cuadrado + resize ----------
def bbox_from_mask(mask, margin=0.10):
    ys, xs = np.where(mask > 0)
    if len(xs) == 0 or len(ys) == 0:
        return None
    x1, x2 = xs.min(), xs.max()
    y1, y2 = ys.min(), ys.max()
    h, w = mask.shape
    # margen relativo
    dx = int((x2 - x1 + 1) * margin)
    dy = int((y2 - y1 + 1) * margin)
    x1 = max(0, x1 - dx); x2 = min(w - 1, x2 + dx)
    y1 = max(0, y1 - dy); y2 = min(h - 1, y2 + dy)
    return x1, y1, x2, y2

def reflect_pad_to_square(img_np):
    """
    Rellena por reflect hasta cuadrado, sin bordes negros.
    img_np: RGB uint8 (H, W, 3)
    """
    h, w = img_np.shape[:2]
    if h == w:
        return img_np
    s = max(h, w)
    pad_top = (s - h) // 2
    pad_bottom = s - h - pad_top
    pad_left = (s - w) // 2
    pad_right = s - w - pad_left
    return cv2.copyMakeBorder(
        img_np, pad_top, pad_bottom, pad_left, pad_right,
        borderType=cv2.BORDER_REFLECT_101
    )

def crop_by_mask_reflect_resize(rgb, mask, out_size=(224, 224)):
    """
    - recorta al bbox de la máscara (+margen interno en bbox_from_mask)
    - reflect-pad hasta cuadrado
    - resize final
    """
    bbox = bbox_from_mask(mask, margin=0.10)
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        crop = rgb[y1:y2+1, x1:x2+1, :]
        if crop.size == 0:
            crop = rgb
    else:
        crop = rgb
    sq = reflect_pad_to_square(crop)
    # resize antialias
    pil = Image.fromarray(sq)
    pil = pil.resize(out_size, Image.Resampling.LANCZOS)
    return np.asarray(pil)

# ---------- 5) Transform clase: todo junto y configurable ----------
class DermPreproc:
    """
    Preprocesado dermatoscópico para usar como transform inicial (antes de ToTensor):
      - exif transpose
      - hair removal (opcional)
      - color constancy Shades-of-Gray (opcional)
      - máscara rápida + crop por máscara + reflect pad
      - resize final
    Devuelve PIL.Image RGB.
    """
    def __init__(self,
                 out_size=(224, 224),
                 do_hair_removal=True,
                 do_color_constancy=True,
                 return_mask=False):
        self.out_size = out_size
        self.do_hair_removal = do_hair_removal
        self.do_color_constancy = do_color_constancy
        self.return_mask = return_mask

    def __call__(self, pil_img):
        # 1) Exif transpose y asegurar RGB
        pil_img = ImageOps.exif_transpose(pil_img)
        if pil_img.mode != "RGB":
            pil_img = pil_img.convert("RGB")

        rgb = np.array(pil_img)

        # 2) Hair removal
        if self.do_hair_removal:
            try:
                rgb = hair_removal_bh_inpaint(rgb, se_size=9, thresh_percentile=90,
                                              dilate_iter=1, method=cv2.INPAINT_TELEA)
            except Exception:
                # Si falla OpenCV o algo, seguimos sin HR
                pass

        # 3) Color constancy (Shades-of-Gray)
        if self.do_color_constancy:
            rgb = shades_of_gray_cc(rgb, p=6)

        # 4) Máscara rápida de lesión
        try:
            mask = quick_lesion_mask(rgb, min_obj=300, hole_area=800)
        except Exception:
            # Fallback: máscara vacía (sin crop)
            mask = np.zeros(rgb.shape[:2], dtype=np.uint8)

        # 5) Crop + reflect pad + resize
        rgb_out = crop_by_mask_reflect_resize(rgb, mask, out_size=self.out_size)

        pil_out = Image.fromarray(rgb_out)
        if self.return_mask:
            # Redimensionamos la máscara al mismo tamaño para inspección/uso
            m = Image.fromarray((mask*255).astype(np.uint8))
            m = m.resize(self.out_size, Image.Resampling.NEAREST)
            return pil_out, m
        return pil_out

# ---------- 6) Ejemplo de uso rápido ----------
if __name__ == "__main__":
    #Ruta de ejemplo (cámbiala por una tuya)
    from pathlib import Path
    img_path = Path("data/raw/train/Benign/8.jpg")
    pil = Image.open(img_path)
    pre = DermPreproc(out_size=(224,224), do_hair_removal=True, do_color_constancy=True, return_mask=True)
    img_proc, mask_proc = pre(pil)
    img_proc.show(); mask_proc.show()
    pass
